In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from time import sleep
import re


Load web pages links file

In [ ]:
df = pd.read_csv('furniture stores pages.csv', names=['Links'], skiprows=1)
links = df['Links'].tolist()

In [ ]:
# RegEx pattern to match 'product' with any words and slashes in front or back
pattern = re.compile(r'[-_\w/]*product[-_\w/]*')

In [ ]:
pages_crawled = 0
data = []
number_of_pages = df.shape[0]

In [ ]:
for url in links:
    if pages_crawled >= number_of_pages:
        break  # Stop crawling

    try:
        response = requests.get(url)
        if response.status_code == 200:
            page_html = response.text
            page_bs = bs(page_html, 'html.parser')

            # Find elements with a class attribute that matches the pattern
            target_tags = page_bs.find_all(lambda tag: tag.name in ['h1', 'h2'] and tag.has_attr('class') and pattern.search(' '.join(tag['class'])))

            # Extract the text content of the found elements
            for tag in target_tags:
                element_text = tag.get_text()
                #data.extend(element_text)
                element_strip = element_text.strip()

                # TODO: check string for proper results
                if "".__eq__(element_strip) or len(element_strip) > 50:
                  data.append([element_strip, 'text', url])
                else:
                  data.append([element_strip, 'product', url])

            # Increment counter
            pages_crawled += 1

            print(f'Crawled page {pages_crawled}: {url}')

        else:
            print(f'Failed to crawl {url}. Status code: {response.status_code}')

    except Exception as e:
        print(f'An error occurred while crawling {url}: {str(e)}')

    sleep(3)

Crawled page 1: https://www.factorybuys.com.au/products/euro-top-mattress-king
Crawled page 2: https://dunlin.com.au/products/beadlight-cirrus
Crawled page 3: https://themodern.net.au/products/hamar-plant-stand-ash


In [ ]:
# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['product', 'match', 'url'])

# Save the data to csv file
extracted_data = 'products_data.csv'
df.to_csv(extracted_data, index=False)